In [ ]:
import os

import numpy as np
import xarray as xr
import xrspatial
from xrspatial.terrain import generate_terrain
import rioxarray
import datashader as ds
from azure.storage.blob import BlobClient
from azure.storage.blob import ContentSettings

In [ ]:
# example with terrain and add 1 band
cvs = ds.Canvas(x_range=(-20,20), y_range=(-20,20), plot_height=400, plot_width=800)
terrain = generate_terrain(canvas=cvs)
terrain = terrain.assign_coords(band = np.asarray(1, np.int64))
terrain = terrain.expand_dims('band')
terrain.dims

In [ ]:
terrain.rio.to_raster('rioxarray_terrain.tiff')

In [ ]:
blob = BlobClient("https://xarrayspatial.blob.core.windows.net/", 
                  "examples-data", 
                  "rioxarray_cog.tiff",
                 credential=os.environ['AZURE_ACCESS_KEY'])
with open('rioxarray_terrain.tiff', 'rb') as img:
    blob.upload_blob(img)
content_settings = ContentSettings(content_type='image/tiff')
blob.set_http_headers(content_settings=content_settings)